In [1]:
# Gradient Clipping in Keras
#<ol>1. Loading the dataset</ol>
## 1. Loading the dataset
# import necessary libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# importing layers from keras
from keras.layers import Dense, InputLayer
from keras.models import Sequential
# importing adam optimizer from keras optimizer module 
from keras.optimizers import Adam

# train_test_split to create training and validation set
from sklearn.model_selection import train_test_split
# accuracy_score to calculate the accuracy of predictions
from sklearn.metrics import accuracy_score


In [2]:
# reading the csv file
data = pd.read_csv('../data/Dataset/emergency_classification.csv')

print(data.head())

  image_names  emergency_or_not
0       0.jpg                 1
1       1.jpg                 1
2       2.jpg                 1
3       3.jpg                 1
4       4.jpg                 1


In [3]:
# create random number generator
seed = 42
rng = np.random.RandomState(seed)


In [5]:
# empty list to store the images
X = []
# iterating over each image
for img_name in data.image_names:
    # loading the image using its name
    img = plt.imread('../data/Dataset/images/' + img_name)
    # saving each image in the list
    X.append(img)


In [6]:
# converting the list of images into array
X=np.array(X)

# storing the target variable in separate variable
y = data.emergency_or_not.values

## 2. Pre-processing the data
# converting 3 dimensional image to 1 dimensional image
X = X.reshape(X.shape[0], 224*224*3)
X.shape
print(X.shape)


(2352, 150528)


In [7]:
# normalizing the pixel values
X = X / X.max()

# minimum and maximum pixel values of images after normalizing
X.min(), X.max()

print(X.min(), X.max())


0.0 1.0


In [8]:
## 3. Creating training and validation set
# creating a training and validation set
X_train, X_valid, y_train, y_valid=train_test_split(X,y,test_size=0.3, random_state=seed)

# shape of training and validation set
(X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape)

print((X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape))


((1646, 150528), (1646,)) ((706, 150528), (706,))


In [9]:
## 4. Defining the model architecture
# defining the model architecture with dropout layer
model=Sequential()

model.add(InputLayer(input_shape=(224*224*3,)))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))


2024-02-04 10:32:51.139056: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-04 10:32:51.139122: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-04 10:32:51.139137: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-04 10:32:51.139509: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-04 10:32:51.139783: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
## 5. Compiling the model
# defining the adam optimizer and setting the learning rate as 10^-5 and adding clip value to clip the gradients
adam = Adam(lr=1e-5, clipvalue=1)


In [11]:
# defining loss as binary crossentropy
# defining optimizer as Adam
# defining metrics as accuracy

model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])



In [12]:
## 6. Training the model
# training the model for 100 epochs

model_history = model.fit(X_train, y_train, epochs=100, batch_size=128,validation_data=(X_valid,y_valid))



Epoch 1/100


2024-02-04 10:37:01.234778: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-02-04 10:37:01.269818: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


13/13 [==============================] - 6s 364ms/step - loss: 0.6977 - accuracy: 0.5729 - val_loss: 0.6869 - val_accuracy: 0.5609
Epoch 2/100
13/13 [==============================] - 1s 83ms/step - loss: 0.6788 - accuracy: 0.5863 - val_loss: 0.6900 - val_accuracy: 0.5609
Epoch 3/100
13/13 [==============================] - 1s 78ms/step - loss: 0.6787 - accuracy: 0.5863 - val_loss: 0.6860 - val_accuracy: 0.5609
Epoch 4/100
13/13 [==============================] - 1s 78ms/step - loss: 0.6787 - accuracy: 0.5863 - val_loss: 0.6895 - val_accuracy: 0.5609
Epoch 5/100
13/13 [==============================] - 1s 80ms/step - loss: 0.6797 - accuracy: 0.5863 - val_loss: 0.6862 - val_accuracy: 0.5609
Epoch 6/100
13/13 [==============================] - 1s 78ms/step - loss: 0.6795 - accuracy: 0.5863 - val_loss: 0.6875 - val_accuracy: 0.5609
Epoch 7/100
13/13 [==============================] - 1s 79ms/step - loss: 0.6802 - accuracy: 0.5863 - val_loss: 0.6857 - val_accuracy: 0.5609
Epoch 8/100
13/13

In [ ]:

## 7. Evaluating model performance 
prediction = model.predict(X_valid)
prediction = prediction.reshape(706,)